<a href="https://colab.research.google.com/github/gangonepriyadarshini/DataVizandEDA/blob/main/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## **Introduction to Colab and Python**

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as s
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import style
import statsmodels.api as sm
import sklearn.tree as tree
import sklearn.model_selection as model_selection
from sklearn.tree import DecisionTreeRegressor 
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
import io


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data=pd.read_csv('User_product_purchase_details_p2.csv')

In [3]:
data2=pd.read_csv('userinfo.csv')

In [4]:
datamerge=pd.merge(data,data2,how='inner',on='User_ID')

In [5]:
datamerge['Product_Category_2']=datamerge['Product_Category_2'].fillna(0)
datamerge['Product_Category_3']=datamerge['Product_Category_3'].fillna(0)

In [6]:
def City_bkt(x):
    if x=='A':
        return 0
    elif x=='B':
        return 1
    else:
        return 2    
datamerge['City_Bkt']=datamerge['City_Category'].map(City_bkt)
def Gender_bkt(x):
    if x=='Male':
        return 1
    else:
        return 0 
datamerge['Gender_Bkt']=datamerge['Gender'].map(Gender_bkt)
def StayYears_bkt(x):
    if x=='0':
        return 'new'
    elif x=='1':
        return 1
    elif x=='2':
        return 2
    elif x=='3':
        return 3
    else:
        return 'morethan4years'
datamerge['StayInYearsBKT']=datamerge['Stay_In_Current_City_Years'].map(StayYears_bkt)
d=datamerge.drop(['User_ID','Product_ID','City_Category','Gender','Stay_In_Current_City_Years'],axis=1)

In [8]:
d.columns

Index(['Marital_Status', 'Product_Category_1', 'Product_Category_2',
       'Product_Category_3', 'Purchase', 'Age', 'Occupation', 'City_Bkt',
       'Gender_Bkt', 'StayInYearsBKT'],
      dtype='object')

In [7]:
d=pd.get_dummies(d)
d['Product_Category_2']=d['Product_Category_2'].astype(int)
d['Product_Category_3']=d['Product_Category_3'].astype(int)

In [8]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550068 entries, 0 to 550067
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype
---  ------                         --------------   -----
 0   Marital_Status                 550068 non-null  int64
 1   Product_Category_1             550068 non-null  int64
 2   Product_Category_2             550068 non-null  int64
 3   Product_Category_3             550068 non-null  int64
 4   Purchase                       550068 non-null  int64
 5   Occupation                     550068 non-null  int64
 6   City_Bkt                       550068 non-null  int64
 7   Gender_Bkt                     550068 non-null  int64
 8   Age_0-17                       550068 non-null  uint8
 9   Age_18-25                      550068 non-null  uint8
 10  Age_26-35                      550068 non-null  uint8
 11  Age_36-45                      550068 non-null  uint8
 12  Age_46-50                      550068 non-null  uint8
 13 

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(d.drop('Purchase',axis = 1),d['Purchase'],test_size=0.2,random_state=111)

In [10]:
xtrain = sm.add_constant(xtrain)
xtest = sm.add_constant(xtest)

In [11]:
from sklearn.ensemble import BaggingRegressor

In [30]:
reg_bag=BaggingRegressor(base_estimator=DecisionTreeRegressor(),
                     n_estimators=90,
                     oob_score=True,
                     n_jobs=-1,random_state=123) 

In [31]:
reg_bag.fit(xtrain,ytrain)

BaggingRegressor(base_estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                      criterion='mse',
                                                      max_depth=None,
                                                      max_features=None,
                                                      max_leaf_nodes=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      presort='deprecated',
                                                      random_state=None,
                                                      splitter='best'),
                 bootstrap=T

In [32]:
reg_bag.score(xtest,ytest)

0.611619144446228

In [29]:
for i in range(100,200,30): #10,30,50,70...trees
    reg_bag=BaggingRegressor(base_estimator=DecisionTreeRegressor(),
                         n_estimators=i,oob_score=True,n_jobs=-1,random_state=123)
    reg_bag.fit(xtrain,ytrain)
    oob=reg_bag.oob_score_
    print('For n_estimators = '+str(i))
    print('OOB Score is '+str(oob))
    print('*******')

For n_estimators = 100
OOB Score is 0.6130098232202371
*******
For n_estimators = 130
OOB Score is 0.6138495008739573
*******
For n_estimators = 160
OOB Score is 0.6144009472729473
*******
For n_estimators = 190
OOB Score is 0.6147340324481662
*******


In [12]:
for i in range(200,300,30): #10,30,50,70...trees
    reg_bag=BaggingRegressor(base_estimator=DecisionTreeRegressor(),
                         n_estimators=i,oob_score=True,n_jobs=-1,random_state=123)
    reg_bag.fit(xtrain,ytrain)
    oob=reg_bag.oob_score_
    print('For n_estimators = '+str(i))
    print('OOB Score is '+str(oob))
    print('*******')

TerminatedWorkerError: ignored

Welcome to this Colab where you will get a quick introduction to the Python programming language and the environment used for the course's exercises: Colab.

Colab is a Python development environment that runs in the browser using Google Cloud.

For example, to print "Hello World", just hover the mouse over [ ] and press the play button to the upper left. Or press shift-enter to execute.

In [ ]:
print("Hello World")

## Functions, Conditionals, and Iteration
Let's create a Python function, and call it from a loop.

In [ ]:
def HelloWorldXY(x, y):
  if (x < 10):
    print("Hello World, x was < 10")
  elif (x < 20):
    print("Hello World, x was >= 10 but < 20")
  else:
    print("Hello World, x was >= 20")
  return x + y

for i in range(8, 25, 5):  # i=8, 13, 18, 23 (start, stop, step)
  print("--- Now running with i: {}".format(i))
  r = HelloWorldXY(i,i)
  print("Result from HelloWorld: {}".format(r))

In [ ]:
print(HelloWorldXY(1,2))

Easy, right?

If you want a loop starting at 0 to 2 (exclusive) you could do any of the following

In [ ]:
print("Iterate over the items. `range(2)` is like a list [0,1].")
for i in range(2):
  print(i)

print("Iterate over an actual list.")
for i in [0,1]:
  print(i)

print("While works")
i = 0
while i < 2:
  print(i)
  i += 1

In [ ]:
print("Python supports standard key words like continue and break")
while True:
  print("Entered while")
  break

## Numpy and lists
Python has lists built into the language.
However, we will use a library called numpy for this.
Numpy gives you lots of support functions that are useful when doing Machine Learning.

Here, you will also see an import statement. This statement makes the entire numpy package available and we can access those symbols using the abbreviated 'np' syntax.

In [ ]:
import numpy as np  # Make numpy available using np.

# Create a numpy array, and append an element
a = np.array(["Hello", "World"])
a = np.append(a, "!")
print("Current array: {}".format(a))
print("Printing each element")
for i in a:
  print(i)

print("\nPrinting each element and their index")
for i,e in enumerate(a):
  print("Index: {}, was: {}".format(i, e))

In [ ]:
print("\nShowing some basic math on arrays")
b = np.array([0,1,4,3,2])
print("Max: {}".format(np.max(b)))
print("Average: {}".format(np.average(b)))
print("Max index: {}".format(np.argmax(b)))

In [ ]:
print("\nYou can print the type of anything")
print("Type of b: {}, type of b[0]: {}".format(type(b), type(b[0])))

In [ ]:
print("\nUse numpy to create a [3,3] dimension array with random number")
c = np.random.rand(3, 3)
print(c)

In [ ]:
print("\nYou can print the dimensions of arrays")
print("Shape of a: {}".format(a.shape))
print("Shape of b: {}".format(b.shape))
print("Shape of c: {}".format(c.shape))
print("...Observe, Python uses both [0,1,2] and (0,1,2) to specify lists")

## Colab Specifics

Colab is a virtual machine you can access directly. To run commands at the VM's terminal, prefix the line with an exclamation point (!).


In [ ]:
print("\nDoing $ls on filesystem")
!ls -l
!pwd

In [ ]:
print("Install numpy")  # Just for test, numpy is actually preinstalled in all Colab instances
!pip install numpy

**Exercise**

Create a code cell underneath this text cell and add code to:


*   List the path of the current directory (pwd)
* Go to / (cd) and list the content (ls -l)

In [ ]:
!pwd
!cd /
!ls -l
print("Hello")

All usage of Colab in this course is completely free or charge. Even GPU usage is provided free of charge for some hours of usage every day.

**Using GPUs**
* Many of the exercises in the course executes more quickly by using GPU runtime: Runtime | Change runtime type | Hardware accelerator | GPU

**Some final words on Colab**
*   You execute each cell in order, you can edit & re-execute cells if you want
*   Sometimes, this could have unintended consequences. For example, if you add a dimension to an array and execute the cell multiple times, then the cells after may not work. If you encounter problem reset your environment:
  *   Runtime -> Restart runtime... Resets your Python shell
  *   Runtime -> Restart all runtimes... Will reset the Colab image, and get you back to a 100% clean environment
* You can also clear the output in the Colab by doing: Edit -> Clear all outputs
* Colabs in this course are loaded from GitHub. Save to your Google Drive if you want a copy with your code/output: File -> Save a copy in Drive...

**Learn More**
*   Check out [this](https://www.youtube.com/watch?v=inN8seMm7UI&list=PLQY2H8rRoyvwLbzbnKJ59NkZvQAW9wLbx&index=3) episode of #CodingTensorFlow, and don't forget to subscribe to the YouTube channel ;)
